In [1]:
# this is my util

In [9]:
from contextlib import contextmanager
import time
import json
import requests
import socket
import sys
import os

In [17]:
# os.system('jupyter nbconvert --to script myutil.ipynb')

# my context

In [3]:
@contextmanager
def gc2():
    gc.enable()
    yield
    gc.collect()

In [4]:
@contextmanager
def timer(title='do'):
    t0 = time.time()
    yield
    print("{} - done in {:.4f}s".format(title, time.time() - t0))

# messange

In [5]:
token = "130fe712252513ab163f9453dbe9bdc9efca01eb3a7f1def85a5f6d89dc7c358"
phones = [17621684529]
# def bot(message, phones=phones, token=token):
def ding(message):
    url = 'https://oapi.dingtalk.com/robot/send'
    params = { 'access_token': token }
    headers = { 'Content-Type': 'application/json' }
    data = json.dumps({
        "msgtype": "text",
        "text": {
            "content": message + "\n" +
                       time.strftime('%Y/%m/%d %H:%M:%S') + "\n" +
                       socket.gethostname()},
        "at": {"atMobiles": phones},
    })
    return requests.post(url, headers=headers, params=params, data=data)

# df2md

将df表格按markdown打印出来。

In [3]:
from tabulate import tabulate

def df2md(df):
    print(tabulate(df, tablefmt="pipe", headers="keys"))

# df 数据处理

In [4]:
def df_null_stat(df, thres_null=0.5):
    """
    args:
        - df
        - thres_null, thres to filter cols to drop
    return:
        - df_null, df of null ratios 
        - cols_drop, cols pass thres
    useage:
    df_null, col_drop = df_null_stat(df, 0.5)
    df.drop(columns=col_drop, axis=1,inplace=True)
    """    
    df_null = pd.DataFrame(df.isnull().sum() / df.shape[0])
    df_null.rename(columns={0:'null_ratio'}, inplace=True);
    df_null = df_null.sort_values(by='null_ratio', ascending=False)
    
    num_null = df_null[df_null.null_ratio > thres_null].shape[0]
    print('num of col null ratio>{}: {}'.format(thres_null, num_null))
    cols_drop = list(df_null[df_null.null_ratio > thres_null].index)

    return df_null, cols_drop